<a href="https://colab.research.google.com/github/LeoneFabio/Egocentric-Vision/blob/main/Video-LLaVa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pillow
!pip install bitsandbytes
!pip install accelerate
!pip install transformers
!pip install av
!pip install numpy
!pip install bitsandbytes-cuda111



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 16.2 MB/s eta 0:00:00


In [2]:
from PIL import Image
import numpy as np
import av
from transformers import VideoLlavaProcessor, VideoLlavaForConditionalGeneration, BitsAndBytesConfig
from huggingface_hub import hf_hub_download
import torch
import os
torch.cuda.is_available()  # Dovrebbe restituire True se la GPU è attiva


True

In [3]:
def read_video_pyav(container, indices):

    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


In [5]:

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)
model = VideoLlavaForConditionalGeneration.from_pretrained(
    "LanguageBind/Video-LLaVA-7B-hf",
    quantization_config=quantization_config,
    device_map="auto"
)
# Carica il processore
processor = VideoLlavaProcessor.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")

# Definisci il prompt
prompt = "USER: <video>what do you see? ASSISTANT:"

video_path = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="sample_demo_1.mp4", repo_type="dataset")
#video_path = '/content/sample-5s.mp4'
# Apri il video con PyAV
container = av.open(video_path)

# Campiona uniformemente 8 frame dal video
total_frames = container.streams.video[0].frames
# Campiona solo 2 frame dal video
indices = np.arange(0, total_frames, total_frames / 15).astype(int)  # Riduce i frame
clip = read_video_pyav(container, indices)


# Prepara input e genera risposta
inputs = processor(text=prompt, videos=clip, return_tensors="pt").to(device)
generate_ids = model.generate(**inputs, max_new_tokens=100)
print(processor.batch_decode(generate_ids, skip_special_tokens=True)[0])

RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):
libcudart.so.11.0: cannot open shared object file: No such file or directory

In [6]:
%%bash
git clone https://github.com/PKU-YuanGroup/Video-LLaVA
cd Video-LLaVA
conda create -n videollava python=3.10 -y
conda activate videollava
pip install --upgrade pip  # enable PEP 660 support
pip install -e .
pip install -e ".[train]"
pip install flash-attn --no-build-isolation
pip install decord opencv-python git+https://github.com/facebookresearch/pytorchvideo.git@28fe037d212663c6a24f373b94cc5d478c8c1a1d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Obtaining file:///content/Video-LLaVA
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a

Cloning into 'Video-LLaVA'...
bash: line 3: conda: command not found
bash: line 4: conda: command not found
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.20 requires pydantic>=2.7.0, but you have pydantic 1.10.19 which is incompatible.
google-genai 0.3.0 requires pydantic<3.0.0dev,>=2.0.0, but you have pydantic 1.10.19 which is incompatible.
google-genai 0.3.0 requires websockets<15.0dev,>=13.0, but you have websockets 11.0.3 which is incompatible.
langchain 0.3.12 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.19 which is incompatible.
langchain-core 0.3.25 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.19 which is incompatible.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
sentence-transformers 3.3.1 requires transformers<5.0.0,

In [3]:
!pip install pillow
!pip install bitsandbytes
!pip install accelerate
!pip install transformers
!pip install av
!pip install numpy
!pip install bitsandbytes-cuda111

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
videollava 1.0.0 requires accelerate==0.21.0, but you have accelerate 1.2.1 which is incompatible.
videollava 1.0.0 requires bitsandbytes==0.41.0, but you have bitsandbytes 0.45.0 which is incompatible.
videollava 1.0.0 requires tokenizers<0.14,>=0.12.1, but you have tokenizers 0.21.0 which is incompatible.
videollava 1.0.0 requires transformers==4.31.0, but you have transformers 4.47.1 which is incompatible.


In [ ]:
from PIL import Image
import numpy as np
import av
from transformers import VideoLlavaProcessor, VideoLlavaForConditionalGeneration, BitsAndBytesConfig
from huggingface_hub import hf_hub_download
import torch
import os
torch.cuda.is_available()  # Dovrebbe restituire True se la GPU è attiva

In [5]:
from huggingface_hub import hf_hub_download
def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)
model = VideoLlavaForConditionalGeneration.from_pretrained(
    "LanguageBind/Video-LLaVA-7B-hf",
    quantization_config=quantization_config,
    device_map="auto"
)
# Carica il processore
processor = VideoLlavaProcessor.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")

# Definisci il prompt
prompt = "USER: <video>what do you see? ASSISTANT:"

video_path = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="sample_demo_1.mp4", repo_type="dataset")
#video_path = '/content/sample-5s.mp4'
# Apri il video con PyAV
container = av.open(video_path)

# Campiona uniformemente 8 frame dal video
total_frames = container.streams.video[0].frames
# Campiona solo 2 frame dal video
indices = np.arange(0, total_frames, total_frames / 15).astype(int)  # Riduce i frame
clip = read_video_pyav(container, indices)


# Prepara input e genera risposta
inputs = processor(text=prompt, videos=clip, return_tensors="pt").to(device)
generate_ids = model.generate(**inputs, max_new_tokens=100)
print(processor.batch_decode(generate_ids, skip_special_tokens=True)[0])

RuntimeError: Failed to import transformers.models.video_llava.modeling_video_llava because of the following error (look up to see its traceback):
Name tf.RaggedTensorSpec has already been registered for class tensorflow.python.ops.ragged.ragged_tensor.RaggedTensorSpec.